# 110道python面试题

这个文件是看知乎的[110倒python面试题](https://zhuanlan.zhihu.com/p/54430650)时用来全部实现一遍以增强记忆的notebook。

作者在写这篇文章的时候有一些不完善和小疏忽，这里我做了勘误和重点强调。

### 1、一行代码实现1--100之和

In [1]:
# 注意[a,b)
sum(range(1,101))

5050

### 2、如何在一个函数内部修改全局变量

In [2]:
a = 5
def fn1():
    a = 1
fn1()
print("local:",a)

def fn2():
    global a
    a = 1
fn2()
print("global:",a)

local: 5
global: 1


### 3、列出5个python标准库

os：提供了不少与操作系统相关联的函数

sys: 通常用于命令行参数

re: 正则匹配

math: 数学运算

datetime:处理日期时间

pickle: 序列化对象

json：处理JSON字符串

### 4、字典如何删除键和合并两个字典

In [3]:
dic = {"name":"abc", "age":16}
del dic["name"] # 删除key
dic

{'age': 16}

In [4]:
dic2 = {"name":"def"}
dic.update(dic2) # 合并字典
dic

{'age': 16, 'name': 'def'}

### 5、谈下python的GIL

GIL 是python的全局解释器锁，同一进程中假如有多个线程运行，一个线程在运行python程序的时候会霸占python解释器（加了一把锁即GIL），使该进程内的其他线程无法运行，等该线程运行完后其他线程才能运行。如果线程运行过程中遇到耗时操作，则解释器锁解开，使其他线程运行。所以在多线程中，线程的运行仍是有先后顺序的，并不是同时进行。

多进程中因为每个进程都能被系统分配资源，相当于每个进程有了一个python解释器，所以多进程可以实现多个进程的同时运行，缺点是进程系统资源开销大

任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。

### 6、python实现列表去重的方法

In [5]:
l = [1,2,3,3,3,3,3,3,4]
a = list(set(l)) # 通过集合去重
a

[1, 2, 3, 4]

### 7、fun(\*args,\*\*kwargs)中的\*args,\*\*kwargs什么意思？

用于函数定义参数。不确定会有多少个参数。`*args`表示一个可变数量的参数列表

In [6]:
def fn(a, *args):
    print(args)
fn(1,2,3,4,5,6,7)

(2, 3, 4, 5, 6, 7)


`**kwargs`表示一个字典，记录传参的时候以`key=value`形式传的可变参数

In [7]:
def fn(**kwargs):
    for k,v in kwargs.items(): # 注意：遍历字典需要使用dict.items()/keys()/values()
        print("key:", k, "value:", v)
fn(key1=1, key2="val2")

key: key1 value: 1
key: key2 value: val2


### 8、python2和python3的range（100）的区别

python2返回列表，python3返回迭代器，节约内存

### 9、一句话解释什么样的语言能够用装饰器?

函数可以作为参数传递的语言，可以使用装饰器

### 10、python内建数据类型有哪些

| 分类 | 数据类型 |
| :-: | :-: |
|文本类型|	str |
|数值类型|	int, float, complex |
|序列类型|	list, tuple, range |
|映射类型|	dict |
|集合类型|	set, frozenset |
|布尔类型|	bool
|二进制类型|	bytes, bytearray, memoryview |

> complex: 虚数

> frozenset: 不能更改的set

> bytes和str都是不可变，不能修改数据，操作是生成新的对象

> bytearray是一个可变数组，类似列表

> memoryview: 不需要复制就可以访问对象

### 11、简述面向对象中__new__和__init__区别

+ `__init__`是初始化方法，创建对象后，就立刻被默认调用了，可接收参数
+ `__new__`至少要有一个参数cls，代表当前类，此参数在实例化时由Python解释器自动识别
+ `__new__`必须要有返回值，返回实例化出来的实例，这点在自己实现`__new__`时要特别注意，可以return父类（通过`super(类名, cls).__new__`），或者直接是object的`__new__`出来的实例
+ `__init__`有一个参数self，就是这个`__new__`返回的实例，`__init__`在`__new__`的基础上可以完成一些其它初始化的动作，`__init__`不需要返回值
+ 如果`__new__`创建的是**当前类**的实例，会**自动调用**`__init__`函数，通过return语句里面调用的`__new__`函数的第一个参数是cls来保证是当前类实例
+ 如果`__new__`的参数是其他类的类名（不是cls），那么实际创建返回的就是其他类的实例，**不仅不会调用当前类的`__init__`函数，也不会调用其他类的`__init__`函数**。

In [8]:
class A(object):
    def __init__(self):
        print("in function __init__")
    def __new__(cls):
        print("in function __new__")
        print("id of cls is", id(cls))
        return super(A, cls).__new__(cls)
a = A()
print("id of class A", id(A))
print("id of instance a", id(a))

in function __new__
id of cls is 2187876432424
in function __init__
id of class A 2187876432424
id of instance a 2187882578696


### 12、简述with方法打开处理文件帮我我们做了什么？

打开文件在进行读写的时候可能会出现一些异常状况，如果按照常规的f.open
写法，我们需要try,except,finally，做异常判断，并且文件最终不管遇到什么情况，都要执行finally f.close()关闭文件，**with方法帮我们实现了finally中f.close**

补充：其实with对很多**成对**的操作都能等价于在finally中自动完成，比如释放锁等。

### 13、列表[1,2,3,4,5],请使用map()函数输出[1,4,9,16,25]，并使用列表推导式提取出大于10的数，最终输出[16,25]

In [9]:
l = [1,2,3,4,5]
l = list(map(lambda x: x*x, l)) # 这里适合用匿名函数 lambda params : operation
print(l)
l = [i for i in l if i > 10]
print(l)

[1, 4, 9, 16, 25]
[16, 25]


### 14、python中生成随机整数、随机小数、0--1之间小数方法

随机整数：random.randint(a,b),生成区间内的整数

0-1随机小数：random.random(),括号中不传参

另外可以使用numpy生成随机的ndArray

In [10]:
import numpy as np
import random

print("random int: ", random.randint(1,10))
print("random float between 0-1: ", random.random())
print("a 5d-array: ", np.random.rand(5))
print("a 5d-array of normal distribution: ", np.random.randn(5))

random int:  8
random float between 0-1:  0.2417164452580184
a 5d-array:  [5.12328169e-01 3.60648453e-01 9.73354717e-01 4.55549568e-01
 4.32625135e-04]
a 5d-array of normal distribution:  [ 0.33621185  1.13677036 -0.80892746 -1.30823215 -0.07967001]


### 15、避免转义给字符串加哪个字母表示原始字符串？
r

### 16、`<div class="nam">中国</div>`，用正则匹配出标签里面的内容（“中国”），其中class的类名是不确定的

In [11]:
import re
pat = r"<div class=\".+\">(\w+)</div>"
match = re.match(pat, '<div class="nam">中国</div>')
match.group(1)

'中国'

### 17、python中断言方法举例
assert（）方法，断言成功，则程序继续执行，断言失败，则程序报错

In [12]:
a,b = 5,2
assert(a>3)
print("assert 'a>3' success")
try:
    assert(b>3)
except:
    print("assert 'b>3' fail")

assert 'a>3' success
assert 'b>3' fail


### 18、数据表student有id,name,score,city字段，其中name中的名字可有重复，需要消除重复行,请写sql语句

select distinct name from student

### 19、10个Linux常用命令
+ ls
+ pwd = print word directory
+ cd
+ touch 修改文件的时间戳属性
+ rm
+ mkdir
+ scp
+ rm
+ cp
+ mv
+ cat
+ grep
+ echo
+ more
+ less

### 20、python2和python3区别？列举5个

1、Python3 使用 print 必须要以小括号包裹打印内容，比如 print('hi')

Python2 既可以使用带小括号的方式，也可以使用一个空格来分隔打印内容，比 如 print 'hi'

2、python2 range(1,10)返回列表，python3中返回迭代器，节约内存

3、python2中使用ascii编码，python中使用utf-8编码

4、python2中unicode表示字符串序列，str表示字节序列

python3中str表示字符串序列，byte表示字节序列

5、python2中为正常显示中文，引入coding声明，python3中不需要

6、python2中是raw_input()函数，python3中是input()函数

### 21、列出python中可变数据类型和不可变数据类型，并简述原理
+ 不可变数据类型：数值型、字符串型string和元组tuple

不允许变量的值发生变化，如果改变了变量的值，相当于是新建了一个对象，而对于相同的值的对象，在内存中则只有一个对象（一个地址），如下图用id()方法可以打印对象的id

In [13]:
a = 1
a1 = 1
b = "abc"
b1 = "abc"
c = (1,2)
c1 = (1,2)

print(id(a))
print(id(a1))
print(id(b))
print(id(b1))
print(id(c))
print(id(c1)) # tuple值一样但是id不一样

print("make changes")
a += 1
c += c1
print(id(a))
print(id(c))

140717348856208
140717348856208
2187811255920
2187811255920
2187881833544
2187903686088
make changes
140717348856240
2187882511640


+ 可变数据类型：列表list和字典dict；

允许变量的值发生变化，即如果对变量进行append、+=等这种操作后，只是改变了变量的值，而不会新建一个对象，变量引用的对象的地址也不会变化，不过对于相同的值的不同对象，在内存中则会存在不同的对象，即每个对象都有自己的地址，相当于内存中对于同值的对象保存了多份，这里不存在引用计数，是实实在在的对象。

In [14]:
a = [1,2,3]
print(id(a))
a.append(4)
print(id(a))

d = {'a':1}
print(id(d))
d.update({'b':2})
print(id(d))

2187883382856
2187883382856
2187903762744
2187903762744


### 22、s = "ajldjlajfdljfddd"，去重并从小到大排序输出"adfjl"

In [15]:
s = "ajldjlajfdljfddd"
s = list(set(s))
s.sort()
print("".join(s))

adfjl


### 23、用lambda函数实现两个数相乘

In [16]:
mul = lambda a,b:a*b
print(mul(3,4))

12


### 24、字典根据键从小到大排序

In [17]:
dic={"name":"zs","age":18,"city":"深圳","tel":"1362626627"}
# 通过items和匿名函数转列表进行排序
lis = sorted(dic.items(), key=lambda i:i[0])
dict(lis)

{'age': 18, 'city': '深圳', 'name': 'zs', 'tel': '1362626627'}

### 25、利用collections库的Counter方法统计字符串每个单词出现的次数"kjalfj;ldsjafl;hdsllfdhg;lahfbl;hl;ahlf;h"

In [18]:
from collections import Counter
a = "kjalfj;ldsjafl;hdsllfdhg;lahfbl;hl;ahlf;h"
res = Counter(a)
print(res['a'])
print(res)

4
Counter({'l': 9, ';': 6, 'h': 6, 'f': 5, 'a': 4, 'j': 3, 'd': 3, 's': 2, 'k': 1, 'g': 1, 'b': 1})


### 26、字符串a = "not 404 found 张三 99 深圳"，每个词中间是空格，用正则过滤掉英文和数字，最终输出"张三 深圳"

In [19]:
a = "not 404 Found 张三 99 深圳"
pat = "([^a-zA-Z0-9\s]+)"
l = re.findall(pat, a)
" ".join(l)

'张三 深圳'

### 27、filter方法求出列表所有奇数并构造新列表，a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
filter() 函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表。该接收两个参数，第一个为函数，第二个为序列，序列的每个元素作为参数传递给函数进行判断，然后返回 True 或 False，最后将返回 True 的元素放到新列表

In [20]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = list(filter(lambda x:x%2, a))
b

[1, 3, 5, 7, 9]

### 29、正则re.complie作用
re.compile是将正则表达式编译成一个对象，加快速度，并重复使用

In [21]:
pat = "[1-9]+"
s = "ssa121aas"
print(re.search(pat, s).group(0))
pat = re.compile(pat)
print(re.search(pat, s).group(0))

121
121


### 30、a=（1，）b=(1)，c=("1") 分别是什么类型的数据？

a: tuple; b: int; c: str

In [22]:
a = (1,)
b = (1)
c = ("1")
print(list(map(type, [a,b,c])))

[<class 'tuple'>, <class 'int'>, <class 'str'>]


### 31、两个列表[1,5,7,9]和[2,2,6,8]合并为[1,2,2,5,6,7,8,9]

In [23]:
a = [1,5,7,9]
b = [2,2,6,8]
print(sorted(a+b)) # 用加号
a.extend(b)
print(sorted(a))   # extend方法一个一个加入而不是像append一样全放入

[1, 2, 2, 5, 6, 7, 8, 9]
[1, 2, 2, 5, 6, 7, 8, 9]


### 32、用python删除文件和用linux命令删除文件方法

python：os.remove(文件名)

linux: rm 文件名

### 33、log日志中，我们需要用时间戳记录error,warning等的发生时间，请用datetime模块打印当前时间戳 “2018-04-01 11:38:54”

In [24]:
import datetime
a = str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print(a)
a = "Week day on " + str(datetime.datetime.now().isoweekday())
print(a)

2020-05-05 17:47:20
Week day on 2


### 34、数据库优化查询方法

外键、索引、联合查询、选择特定字段等等

### 35、请列出你会的任意一种统计图（条形图、折线图等）绘制的开源库，第三方也行

pyecharts、matplotlib

### 36、写一段自定义异常代码

In [25]:
def fn(x):
    if(x>2):
        raise Exception("x is greater than 2!")
try:
    fn(3)
except Exception as e:
    print(e)
finally:
    print("finally")

x is greater than 2!
finally


### 37、正则表达式匹配中，（.*）和（.*?）匹配区别？

（.*）是贪婪匹配，会把满足正则的尽可能多的往后匹配

（.*?）是非贪婪匹配，会把满足正则的尽可能少匹配

### 38、简述Django的orm
ORM，全拼Object-Relation Mapping，意为对象-关系映射

实现了数据模型与数据库的解耦，通过简单的配置就可以轻松更换数据库，而不需要修改代码只需要面向对象编程,orm操作本质上会根据对接的数据库引擎，翻译成对应的sql语句,所有使用Django开发的项目无需关心程序底层使用的是MySQL、Oracle、sqlite....，如果数据库迁移，只需要更换Django的数据库引擎即可

### 39、[[1,2],[3,4],[5,6]]一行代码展开该列表，得出[1,2,3,4,5,6]

In [26]:
a = [[1,2],[3,4],[5,6]]
[j for i in a for j in i]

[1, 2, 3, 4, 5, 6]

In [27]:
np.array(a).flatten().tolist()

[1, 2, 3, 4, 5, 6]

### 40、x="abc",y="def",z=["d","e","f"],分别求出x.join(y)和x.join(z)返回的结果

dabceabcf
dabceabdf

In [28]:
x="abc"
y="def"
z=["d","e","f"]
x.join(y)

'dabceabcf'

In [29]:
x.join(z)

'dabceabcf'

### 41、举例说明异常模块中try except else finally的相关意义

try..except..else没有捕获到异常，执行else语句

try..except..finally不管是否捕获到异常，都执行finally语句

### 42、python中交换两个数值

```python
a, b = b, a
```

### 43、举例说明zip()函数用法
zip()函数在运算时，会以一个或多个序列（可迭代对象）做为参数，返回一个元组的列表。同时将这些序列中并排的元素配对。

zip()参数可以接受任何类型的序列，同时也可以有两个以上的参数;当传入参数的长度不同时，zip能自动以最短序列长度为准进行截取，获得元组。

In [30]:
a = [1,2,3]
b = ['a','b','c']
c = "小猪佩奇"
list(zip(a,b,c))

[(1, 'a', '小'), (2, 'b', '猪'), (3, 'c', '佩')]

### 44、a="张明 98分"，用re.sub，将98替换为100

In [31]:
pat = '\d+'
re.sub(pat,'100', "张明 98分")

'张明 100分'

### 45、写5条常用sql语句

show databases;

show tables;

desc 表名;

select * from 表名;

delete from 表名 where id=5;

update students set gender=0,hometown="北京" where id=5

grant all privileges on somebody

### 46、a="hello"和b="你好"编码成bytes类型

In [32]:
a=b"hello"
b="你好".encode()
print(a,b)
print(list(map(type,[a,b])))

b'hello' b'\xe4\xbd\xa0\xe5\xa5\xbd'
[<class 'bytes'>, <class 'bytes'>]


### 47、[1,2,3]+[4,5,6]的结果是多少？

[1,2,3,4,5,6]

### 48、提高python运行效率的方法

1、使用生成器，因为可以节约大量内存

2、循环代码优化，避免过多重复代码的执行

3、核心模块用Cython PyPy等，提高效率

4、多进程、多线程、协程

5、多个if elif条件判断，可以把最有可能先发生的条件放到前面写，这样可以减少程序判断的次数，提高效率

In [33]:
#### 生成器
g = (i*i for i in range(1, 11)) # 生成式方式创建
a = next(g)
print(a)
for i in g:
    print(i, end=' ') # 迭代和next都是一次性的
for i in g:
    print("已经next到最后了，g已经空了，这条不会打印")

1
4 9 16 25 36 49 64 81 100 

In [34]:
def fn(max): # 函数形式生成器
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a+b
        n += 1
# 函数执行到yield语句暂时返回，next()方法可以再次进入函数，从yield语句继续

a = fn(10)
for i in a:
    print(i, end=' ')

1 1 2 3 5 8 13 21 34 55 

### 49、简述mysql和redis区别

redis： 内存型非关系数据库，数据保存在内存中，速度快

mysql：关系型数据库，数据保存在磁盘中，检索的话，会有一定的Io操作，访问速度相对慢

### 50、遇到bug如何处理

### 51、正则匹配，匹配日期2018-03-20

### 52、list=[2,3,5,4,9,6]，从小到大排序，不许用sort，输出[2,3,4,5,6,9]

### 53、写一个单列模式

In [41]:
import threading

class Singleton:
    __instance = None
    
    lock = threading.Lock() # 类锁
    
    def __new__(cls, name):
        if not cls.__instance:
            with Singleton.lock:
                if not cls.__instance:
                    cls.__instance = object.__new__(cls)
        return cls.__instance
    
    def __init__(self, name):
        if not hasattr(self, 'name'):
            with Singleton.lock:
                if not hasattr(self, 'name'):
                    self.name = name
        
lock = None
a = Singleton("a")
b = Singleton("b")
print(id(a), id(b))
print(a.name, b.name)

2187904584776 2187904584776
a a


In [42]:
def instanceIt(s):
    a = Singleton(s)
    print("Name in (%s) is %s" % (s, a.name))
    print("Instance ID is %s" % id(a))

t1 = threading.Thread(target=instanceIt,args=("t1",))
t2 = threading.Thread(target=instanceIt,args=("t2",))

t1.start()
t2.start()

Name in (t1) is a
Instance ID is 2187904584776
Name in (t2) is a
Instance ID is 2187904584776


### 54、保留两位小数

In [43]:
a = 1.3335
round(float(a),2)

1.33

### 55、求三个方法打印结果

In [44]:
def fn(k,v,dic={}):
    dic[k] = v
    print(dic)
fn("one",1)
fn("two",2) # 默认的引用对象依然存在
fn("three",3,{})

{'one': 1}
{'one': 1, 'two': 2}
{'three': 3}


### 56、列出常见的状态码和意义

200 OK

请求正常处理完毕

204 No Content

请求成功处理，没有实体的主体返回

206 Partial Content

GET范围请求已成功处理

301 Moved Permanently

永久重定向，资源已永久分配新URI

302 Found

临时重定向，资源已临时分配新URI

303 See Other

临时重定向，期望使用GET定向获取

304 Not Modified

发送的附带条件请求未满足

307 Temporary Redirect

临时重定向，POST不会变成GET

400 Bad Request

请求报文语法错误或参数错误

401 Unauthorized

需要通过HTTP认证，或认证失败

403 Forbidden

请求资源被拒绝

404 Not Found

无法找到请求资源（服务器无理由拒绝）

500 Internal Server Error

服务器故障或Web应用故障

503 Service Unavailable

服务器超负载或停机维护

### 57、分别从前端、后端、数据库阐述web项目的性能优化

该题目网上有很多方法，我不想截图网上的长串文字，看的头疼，按我自己的理解说几点

#### 前端优化：

1、减少http请求、例如制作精灵图

2、html和CSS放在页面上部，javascript放在页面下面，因为js加载比HTML和Css加载慢，所以要优先加载html和css,以防页面显示不全，性能差，也影响用户体验差



#### 后端优化：

1、缓存存储读写次数高，变化少的数据，比如网站首页的信息、商品的信息等。应用程序读取数据时，一般是先从缓存中读取，如果读取不到或数据已失效，再访问磁盘数据库，并将数据再次写入缓存。

2、异步方式，如果有耗时操作，可以采用异步，比如celery

3、代码优化，避免循环和判断次数太多，如果多个if else判断，优先判断最有可能先发生的情况



#### 数据库优化：

1、如有条件，数据可以存放于redis，读取速度快

2、建立索引、外键等



### 58、使用pop和del删除字典中的"name"字段，dic={"name":"zs","age":18}

In [45]:
dic={"name":"zs","age":18}
dic.pop("name")
dic

{'age': 18}

In [46]:
dic={"name":"zs","age":18}
del dic["name"]
dic

{'age': 18}

### 59、列出常见MYSQL数据存储引擎

InnoDB：支持**事务处理**，支持**外键**，支持崩溃修复能力和**并发控制**。如果需要对事务的完整性要求比较高（比如银行），要求实现并发控制（比如售票），那选择InnoDB有很大的优势。如果需要频繁的更新、删除操作的数据库，也可以选择InnoDB，因为支持事务的提交（commit）和回滚（rollback）。

MyISAM：**插入数据快**，**空间和内存使用比较低**。如果表主要是用于插入新记录和读出记录，那么选择MyISAM能实现处理高效率。如果应用的完整性、并发性要求比 较低，也可以使用。

MEMORY：所有的数据都在内存中，数据的处理速度快，但是安全性不高。如果需要很快的读写速度，对数据的安全性要求较低，可以选择MEMOEY。它对表的大小有要求，不能建立太大的表。所以，这类数据库只使用在相对较小的数据库表。

### 60、dict()创建字典新方法
通过传入列表或元组的列表

In [49]:
s1 = dict([("k1","v1"),("k2","v2")])
s2 = dict([["k1","v1"],["k2","v2"]])
print(s1,s2)

{'k1': 'v1', 'k2': 'v2'} {'k1': 'v1', 'k2': 'v2'}


### 61、简述同源策略
同源策略（same-origin policy）是一种出于浏览器安全方面的考虑而出台的一种策略，它可以保证用户信息的安全，防止恶意的网站窃取。同源策略只允许与本域下的接口交互，不同源的客户端脚本在没有明确授权的情况下，不能读写对方的资源。

同源策略需要同时满足以下三点要求：

1）协议相同

2）域名相同

3）端口相同

http:http://www.test.com与https:http://www.test.com 不同源——协议不同

http:http://www.test.com与http:http://www.admin.com 不同源——域名不同

http:http://www.test.com与http:http://www.test.com:8081 不同源——端口不同

只要不满足其中任意一个要求，就不符合同源策略，就会出现“跨域”

#### 如何跨域？
由于同源策略的影响，当我们从一个域名的网页去请求另一个域名的资源时，就无法成功获取资源。如果我们要想成功获取资源，那么就要用到跨域。

+ JSONP 通过后端获取数据
```html
<script src="http://117.0.0.1:8080/getNews?callback=showData"></script>
```
+ CORS 全称是跨域资源共享，是一种ajax跨域请求资源的方式

当发生跨域时，浏览器会在请求头中加入Origin字段，值为源站url；响应头中有Access-Control-Allow-Origin字段，内容是允许跨域访问的源url。如果ACAO中有O的值，则浏览器会发送，否则浏览器会驳回请求。

### 62、简述cookie和session的区别

1，session 在服务器端，cookie 在客户端（浏览器）

2、session 的运行依赖 session id，而 session id 是存在 cookie 中的，也就是说，如果浏览器禁用了 cookie ，同时 session 也会失效，存储Session时，键与Cookie中的sessionid相同，值是开发人员设置的键值对信息，进行了base64编码，过期时间由开发人员设置

3、cookie安全性比session差

### 63、简述多线程、多进程

#### 进程：

1、操作系统进行资源分配和调度的基本单位，多个进程之间相互独立

2、稳定性好，如果一个进程崩溃，不影响其他进程，但是进程消耗资源大，开启的进程数量有限制



#### 线程：

1、CPU进行资源分配和调度的基本单位，线程是进程的一部分，是比进程更小的能独立运行的基本单位，一个进程下的多个线程可以共享该进程的所有资源

2、如果IO操作密集，则可以多线程运行效率高，缺点是如果一个线程崩溃，都会造成进程的崩溃

#### 应用：

IO密集的用多线程，在用户输入，sleep 时候，可以切换到其他线程执行，减少等待的时间

CPU密集的用多进程，因为假如IO操作少，用多线程的话，因为线程共享一个全局解释器锁，当前运行的线程会霸占GIL，其他线程没有GIL，就不能充分利用多核CPU的优势

### 64、简述any()和all()方法

any():只要迭代器中有一个元素为真就为真

all():迭代器中所有的判断项返回都是真，结果才为真

In [50]:
a = [1,"2",(3,),None]
print(any(a), all(a))

True False


### 65、IOError、AttributeError、ImportError、IndentationError、IndexError、KeyError、SyntaxError、NameError分别代表什么异常

IOError：输入输出异常

AttributeError：试图访问一个对象没有的属性

ImportError：无法引入模块或包，基本是路径问题

IndentationError：语法错误，代码没有正确的对齐

IndexError：下标索引超出序列边界

KeyError:试图访问你字典里不存在的键

SyntaxError:Python代码逻辑语法出错，不能执行

NameError:使用一个还未赋予对象的变量

### 66、python中copy和deepcopy区别

1、复制不可变数据类型，不管copy还是deepcopy,和=“赋值”的情况一样，对象的id值与浅复制原来的值相同。

In [52]:
import copy
a = "hello"
b = a
c = copy.copy(a)
d = copy.deepcopy(a)
print(list(map(id,[a,b,c,d])))

[2187938124656, 2187938124656, 2187938124656, 2187938124656]


2、复制的值是可变对象（列表和字典）

浅拷贝copy有两种情况：

第一种情况：复制的 对象中无 复杂 子对象，原来值的改变并不会影响浅复制的值，同时浅复制的值改变也并不会影响原来的值。原来值的id值与浅复制原来的值不同。

第二种情况：复制的对象中有 复杂 子对象 （例如列表中的一个子元素是一个列表）， 改变原来的值 中的复杂子对象的值 ，会影响浅复制的值。

深拷贝deepcopy：完全复制独立，包括内层列表和字典

**也就是说，对于内部复杂对象，浅拷贝是同一个引用，深拷贝是新的对象**

### 67、列出几种魔法方法并简要介绍用途

__init__:对象初始化方法

__new__:创建对象时候执行的方法，单列模式会用到

__str__:当使用print输出对象的时候，只要自己定义了__str__(self)方法，那么就会打印从在这个方法中return的数据

__del__:删除对象执行的方法

__len__:长度

__getitem__:按照idx获取值

### 68、python 1.py 22 33命令行启动程序并传参，print(sys.argv)会输出什么数据？
['1.py','22','33']

### 69、请将[i for i in range(3)]改成生成器
```python
(i for i in range(3))
```

### 70、a = " hehheh ",去除首尾空格

In [55]:
a = " hehheh "
a.strip()

'hehheh'

### 71、举例sort和sorted对列表排序，list=[0,-1,3,-10,5,9]
sort是inplace的，返回None，sorted返回排序后的对象。

### 72、对list排序foo = [-5,8,0,4,9,-4,-20,-2,8,2,-4],使用lambda函数从小到大排序

In [56]:
foo = [-5,8,0,4,9,-4,-20,-2,8,2,-4]
sorted(foo, key=lambda x:x)

[-20, -5, -4, -4, -2, 0, 2, 4, 8, 8, 9]

### 73、使用lambda函数对list排序foo = [-5,8,0,4,9,-4,-20,-2,8,2,-4]，输出结果为[0,2,4,8,8,9,-2,-4,-4,-5,-20]，正数从小到大，负数从大到小

In [ ]:
foo = [-5,8,0,4,9,-4,-20,-2,8,2,-4]
sorted(foo, key=lambda x:(x<0,abs(x)))
# 首先x<0, 正数返回0，负数返回1，这样第一轮排序是正数在前负数在后
# 第二轮排序对每组按照绝对值从小到大排

### 74、列表嵌套字典的排序，分别根据年龄和姓名排序
还是 key
### 75、列表嵌套元组，分别按字母和数字排序
key返回元组
### 76、列表嵌套列表排序，年龄数字相同怎么办？
key
### 77、根据键对字典排序（用zip函数）
### 78、根据键对字典排序（不用zip函数）

In [63]:
dic = {"name":"zs", "sex":"man", "city":"bj"}
foo1 = list(zip(dic.keys(), dic.values())) # zip
foo2 = dic.items() # no zip
print(sorted(foo1, key=lambda x:x[0]))
print(sorted(foo2, key=lambda x:x[0]))
dic = dict(sorted(foo1, key=lambda x:x[0]))
print(dic)

[('city', 'bj'), ('name', 'zs'), ('sex', 'man')]
[('city', 'bj'), ('name', 'zs'), ('sex', 'man')]
{'city': 'bj', 'name': 'zs', 'sex': 'man'}


### 79、列表推导式、字典推导式、生成器
只写字典了

In [65]:
dic = {k:"V"+k for k in "12345"}
dic

{'1': 'V1', '2': 'V2', '3': 'V3', '4': 'V4', '5': 'V5'}

### 80、最后出一道检验题目，根据字符串长度排序，看排序是否灵活运用

In [66]:
s = ['ab','abcd','a','ab']
sorted(s, key=lambda x:len(x))

['a', 'ab', 'ab', 'abcd']

### 81、举例说明SQL注入和解决办法

当以字符串格式化书写方式的时候，如果用户输入的有;+SQL语句，后面的SQL语句会执行，比如例子中的SQL注入会删除数据库demo

In [68]:
good_input = "zs"
fake_input = "zs;drop database demo"

bad_sql = "select * from demo where name = %s" % fake_input
bad_sql

'select * from demo where name = zs;drop database demo'

通过参数化来解决SQL注入。参数化预先编译好SQL语句，留下参数位置。

参数化能防注入的原因在于，语句是语句，参数是参数，参数的值并不是语句的一部分，数据库只按语句的语义跑，至于跑的时候是带一个普通背包还是一个怪物，不会影响行进路线，无非跑的快点与慢点的区别。

### 85、python字典和json字符串相互转化方法

In [69]:
import json
dic = {"name":"zs"}
json.dumps(dic)

'{"name": "zs"}'

In [70]:
s = '{"name": "zs"}'
json.loads(s)

{'name': 'zs'}

### 91、简述python引用计数机制

python垃圾回收主要以引用计数为主，标记-清除和分代清除为辅的机制，其中标记-清除和分代回收主要是为了处理循环引用的难题。
引用计数算法

当有1个变量保存了对象的引用时，此对象的引用计数就会加1

当使用del删除变量指向的对象时，如果对象的引用计数不为1，比如3，那么此时只会让这个引用计数减1，即变为2，当再次调用del时，变为1，如果再调用1次del，此时会真的把对象进行删除